  ## Dataset



  We will be using the CAMELS-GB dataset. This contains daily hydrometeorological data for around 670 catchments in Great Britain, as well as catchment attributes related to land use/land cover, geology, and climate. The data can be downloaded [here](https://catalogue.ceh.ac.uk/documents/8344e4f3-d2ea-44f5-8afa-86d2987543a9). However, I have already placed a copy of the data in the shared data directory for this course. The `SHARED_DATA_DIR` environment variable contains the full path to the shared data directory. In the following code we will create a new path variable so that we can easily navigate to the CAMELS-GB data files:

In [ ]:
import os
from pathlib import Path

SHARED_DATADIR = Path(os.environ["SHARED_DATA_DIR"])
DATADIR = SHARED_DATADIR / '8344e4f3-d2ea-44f5-8afa-86d2987543a9' / 'data'

Traceback (most recent call last):
  File "/Users/smoulds/.vscode/extensions/ms-python.python-2025.20.1-darwin-arm64/python_files/python_server.py", line 134, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 6, in <module>
TypeError: unsupported operand type(s) for /: 'str' and 'str'



  Now Load the data for a catchment chosen at random. The timeseries data are stored as csv files, so we use Pandas to load them into a Pandas DataFrame object:

In [ ]:
import pandas as pd
id = '97002'
data = pd.read_csv(os.path.join(DATADIR, 'timeseries', f'CAMELS_GB_hydromet_timeseries_{id}_19701001-20150930.csv'), parse_dates=[0])
print(data.head())


        date  precipitation   pet  ...  shortwave_rad  longwave_rad  windspeed
0 1970-10-01           9.93  1.02  ...          61.76        325.33       7.65
1 1970-10-02           4.01  1.41  ...          93.56        294.20      10.03
2 1970-10-03           7.27  1.17  ...          61.95        321.14       5.41
3 1970-10-04           3.77  0.06  ...          42.83        341.28       7.27
4 1970-10-05           1.19  1.56  ...          92.13        299.08       7.90

[5 rows x 11 columns]


 Look at the CAMELS-GB [manuscript](https://doi.org/10.5194/essd-12-2459-2020) and find out the units of each variable. Verify that `discharge_spec` is consistent with `discharge_vol` (HINT: you will need to find the drainage area of your chosen catchment so you can convert the volume to a depth. For now, you can find out this information by looking at the [NRFA website](https://nrfa.ceh.ac.uk/data/search). Later we will use the static catchment attributes provided with CAMELS-GB).

In [ ]:
# This is a question of unit conversion. I will convert discharge_spec (mm/d) to volumetric discharge (m3/s). To do this, we need to know the area of the catchment. From the NRFA website for station 97002 we can see that the catchment area is 412.8km2.
data['discharge_vol_computed'] = data['discharge_spec']
data['discharge_vol_computed'] /= 1000 # mm/d -> m/d [1000 mm in 1m]
data['discharge_vol_computed'] *= 412.8 * 1000 * 1000 # m/d -> m3/d [1 km2 = 1000 * 1000 m2]
data['discharge_vol_computed'] /= 86400 # m3/d -> m3/s [1 day = 86400 seconds]
print(data[['discharge_spec', 'discharge_vol', 'discharge_vol_computed']].dropna().head()) # Check approximate equality (note they will not be exactly equal due to rounding)

     discharge_spec  discharge_vol  discharge_vol_computed
457            1.12           5.37                5.351111
458            1.05           5.03                5.016667
459            1.08           5.20                5.160000
460            1.19           5.73                5.685556
461            1.12           5.37                5.351111


  Later on it will be useful to have the catchment ID in the dataframe, so we add it here:

In [ ]:
data['id'] = id


 We can also see that the `discharge_vol` column contains `NaN` values - these usually indicate missing data. There are various things we can do to handle (or impute) missing values, but for now let's just remove them:

In [ ]:
data = data.dropna(subset=['discharge_vol'])
data.head()


  Recall the water balance equation from lecture 1:



  $\frac{dS}{dt} = P - E - Q$



  where $\frac{dS}{dt}$ is the change in storage over time, $P$ is precipitation, $E$ is evaporation and $Q$ is streamflow. Also recall that over long time periods we can assume the storage term tends towards zero. Now we can write:



  $0 = P - E - Q$



  and hence:



  $E = P - Q.$



  This is convenient because evaporation is hard to measure accurately. Let's use the equation above to estimate the catchment-averaged evaporation. We will work at annual timescales so that we can reasonably assume that the storage term is negligible. First we need to compute the annual precipitation and discharge. To do this we typically use the "water year" instead of the calendar year. This avoids the potential for large errors in the water balance because catchment storage can vary significantly during the wet season. In the UK the water year is taken as 1st October to 30th September. Fortunately Pandas has some magic that allows us to easily aggregate by water year:

In [ ]:
data['water_year'] = data['date'].dt.to_period('Y-SEP')
data.head()



Here, `Y-SEP` is a period alias for "annual frequency, anchored end of September". Learn more about period aliases by consulting the [Pandas documentation](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases).

Now we group our dataframe by the new `water_year` column, and compute the sum of precipitation and discharge. Before doing this we need to convert discharge_vol from m3/s (daily average discharge) to m3/day (daily total discharge):

In [ ]:
data['discharge_vol'] *= 60 * 60 * 24 # m3/s -> m3/day
anndata = data.groupby(['id', 'water_year'])[['precipitation', 'pet', 'discharge_spec', 'discharge_vol']].sum().reset_index()

  Aggregating data is an extremely useful skill in hydrology. Think about how you might use Pandas to aggregate by month or by season.







  When making comparisons between catchments, it is common to transform all variables to a *depth* so that the effect of catchment area is reduced. This allows us to compare the hydrological behaviour of a large catchment (e.g. Tweed) with a much smaller catchment. Let's load the catchment attributes and find the area of our catchment.

In [ ]:
metadata = pd.read_csv(os.path.join(DATADIR, f'CAMELS_GB_topographic_attributes.csv'))
metadata['gauge_id'] = metadata['gauge_id'].astype(str)
area = metadata[metadata['gauge_id'] == id]['area'].values[0]
area *= 1e6 # km2 -> m2 



  Let's return to the question I posed above, about verifying that discharge_spec is consistent with discharge_vol. Let's transform our volumetric data to depth units:

In [ ]:
anndata['discharge_spec_computed'] = anndata['discharge_vol'].copy() # m3/day
anndata['discharge_spec_computed'] /= area # m3 -> m
anndata['discharge_spec_computed'] *= 1000 # m -> mm



  If you look at the dataframe, you see that column `discharge_vol` is now the same as `discharge_spec`. In future, you can use `discharge_spec` directly, without the need for transformation. We now have everything we need to estimate evaporation using the water balance equation:

In [ ]:
anndata['diff'] = anndata['discharge_spec'] - anndata['discharge_spec_computed']
print(anndata[['discharge_spec', 'discharge_spec_computed', 'diff']].head())
print(anndata['diff'].abs().mean())


   discharge_spec  discharge_spec_computed      diff
0          275.61               275.705385 -0.095385
1          436.02               436.039838 -0.019838
2          612.14               612.204285 -0.064285
3          661.68               661.747539 -0.067539
4          477.49               477.428373  0.061627
0.05063917987050362


In [ ]:
anndata['evaporation'] = anndata['precipitation'] - anndata['discharge_spec_computed']



  Let's plot this data:

In [ ]:
import matplotlib.pyplot as plt

anndata = anndata.set_index('water_year')
anndata.plot(y=['precipitation', 'discharge_spec', 'evaporation'], figsize=(12, 6))

plt.title(f'Water balance for catchment {id}')
plt.xlabel('Water year end')
plt.ylabel('Depth (mm)')
plt.legend(['Precipitation', 'Discharge', 'Evaporation'])
plt.grid(True)
plt.show()

anndata = anndata.reset_index()


 Instead of estimating evaporation at the annual scale, we could also do this at the monthly or seasonal scale. Here is a helpful code snippet for adding the season to the dataframe:

In [ ]:
def month_to_season(month):
    if month in [12, 1, 2]:
        return 'DJF'
    elif month in [3, 4, 5]:
        return 'MAM'
    elif month in [6, 7, 8]:
        return 'JJA'
    else:
        return 'SON'

# Align year to start in September - this ensures that DJF (which spans two calendar years) is grouped correctly
data['season_year'] = data['date'].dt.to_period('Y-AUG')
data['season'] = data.date.dt.month.apply(month_to_season)
seasondata = data.groupby(['id', 'season_year', 'season'])[['precipitation', 'pet', 'discharge_spec', 'discharge_vol']].sum().reset_index()

# The plot order is not logical at the season scale, so we need to add a couple of extra columns
season_order = {
    'SON': 0,
    'DJF': 1,
    'MAM': 2,
    'JJA': 3
}
seasondata['season_index'] = seasondata['season'].map(season_order)
seasondata['plot_index'] = (
    seasondata['season_year'].dt.year * 4 # multiply by 4 because there are four seasons
    + seasondata['season_index']
)
seasondata = seasondata.sort_values('plot_index')
seasondata['season_label'] = (
    seasondata['season_year'].dt.year.astype(str)
    + '-'
    + seasondata['season']
)
print(seasondata.head())

      id season_year season  ...  season_index  plot_index  season_label
0  97002        1972    DJF  ...             1        7889      1972-DJF
2  97002        1972    MAM  ...             2        7890      1972-MAM
1  97002        1972    JJA  ...             3        7891      1972-JJA
6  97002        1973    SON  ...             0        7892      1973-SON
3  97002        1973    DJF  ...             1        7893      1973-DJF

[5 rows x 10 columns]


 Now you can compute seasonal evaporation in the same way as before. However, this time you will need to use the `plot_index` column for the x-axis. What do you notice about the seasonal evaporation? Are there any seasons where evaporation is negative? What might this imply?

In [ ]:
seasondata['evaporation'] = seasondata['precipitation'] - seasondata['discharge_spec']
seasondata = seasondata.set_index('plot_index')
seasondata.plot(y=['precipitation', 'discharge_spec', 'evaporation'], figsize=(12, 6))
plt.title(f'Water balance for catchment {id}')
plt.xlabel('Water year end')
plt.ylabel('Depth (mm)')
plt.legend(['Precipitation', 'Discharge', 'Evaporation'])
plt.grid(True)
plt.show()

Traceback (most recent call last):
  File "/Users/smoulds/.vscode/extensions/ms-python.python-2025.20.1-darwin-arm64/python_files/python_server.py", line 134, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NotImplementedError



Negative evaporation is impossible. It is indicative of an incorrect assumption that $\frac{dS}{dt} = 0$ at the seasonal scale. Intuitively this makes sense - in the UK, we would expect the catchment to contain more water in its various stores (e.g. groundwater, lakes, reservoirs etc.) at the end of winter compared to the start, because winter is our wettest season. On the other hand, we would expect the catchment to contain less water at the end of summer. In fact the end of summer tends to be the annual minimum in terms of catchment water storage, which is why the start of the water year is taken as the 1 October. 

We can also express this mathematically. Since 

$E = P - Q - \frac{dS}{dt}$ 

Under our (incorrect) assumption that $\frac{dS}{dt} = 0$, when $E < 0$, $Q > P$. This suggests that streamflow is being sustained by water that fell in a previous season - i.e., water is being released from storage (either through natural processes or human intervention) to become streamflow. Note that while $E < 0$ is physically impossible, $\frac{dS}{dt} < 0$ is possible - it means that the change in storage with respect to time is negative - i.e. the stores are emptying rather than filling up. Note also that because evaporation must be greater than or equal to zero, the situation $P < Q$ is not limited to occasions when $E < 0$ in our dataframe above. The reality is that water is constantly moving from the atmosphere, to various catchment water stores, to streamflow. The main point is that the relative magnitude of these processes varies in time.  

  ## Land cover impacts



  We will cover the drivers of evaporation in more detail later on the course. One question we may have is the role of different land cover types on the water balance. Let's investigate whether land use impacts evaporation by looking at some forested catchments:

In [ ]:
metadata_lu = pd.read_csv(os.path.join(DATADIR, f'CAMELS_GB_landcover_attributes.csv'))



  Have a look at the columns in `metadata_lu` and consult Coxon et al. (2020). Which columns represent forest? Create a new column called `forest_perc` that combines the two types.

In [ ]:
metadata_lu['forest_perc'] = metadata_lu[['ewood_perc', 'dwood_perc']].sum(axis=1)
print(metadata_lu[['gauge_id', 'dwood_perc', 'ewood_perc', 'forest_perc']].head())
print(metadata_lu['forest_perc'].describe())

   gauge_id  dwood_perc  ewood_perc  forest_perc
0     10002        3.89        5.41         9.30
1     10003        4.74        3.06         7.80
2      1001        0.41       12.37        12.78
3    101002        6.20        0.30         6.50
4    101005        4.60        0.12         4.72
count    671.000000
mean      13.863949
std       11.641904
min        0.000000
25%        6.340000
50%       10.920000
75%       17.065000
max       92.780000
Name: forest_perc, dtype: float64


  To compare the impact of vegetation on runoff generation, it would be useful to compute a summary measure for each catchment. One such measure, or signature, is the runoff ratio, defined as the proportion of precipitation that becomes runoff. We can calculate this as follows:

In [ ]:
anndata_sum = anndata.groupby('id')[['precipitation', 'discharge_spec_computed']].sum()
anndata_sum['runoff_ratio'] = anndata_sum['discharge_spec_computed'] / anndata_sum['precipitation']
print(f"Runoff ratio for catchment {id}: {anndata_sum['runoff_ratio'].iloc[0]}")

Runoff ratio for catchment 97002: 0.6345938008054475


 What does the runoff ratio tell us about the catchment? Would you expect the number to be higher or lower for an arid catchment? What about a humid catchment?

 ## Optional exercise



 1. Divide the catchments into three groups: Low forest (<10%), Medium forest (10-30%) and High forest (>30%).

 2. Compute the runoff ratio for every catchment (HINT: write a loop to perform the steps outlined above).

 3. Make a boxplot showing the distribution of runoff ratios for each group.

 4. Can you see a clear pattern? Consider the following questions:

    (i)   Is this is a fair comparison (HINT: look at the number of catchments in each group)?

    (ii)  What other factors might be influencing the runoff ratio?

    (iii) How could you improve this analysis? How might statistical or machine learning models help?

First let's divide the catchments into three groups. We can do this by applying a function across rows:

In [ ]:
def forest_group(x):
    if x < 10:
        return "low"
    elif x < 30:
        return "medium"
    else:
        return "high"

metadata_lu["forest_group"] = metadata_lu["forest_perc"].apply(forest_group)
print(metadata_lu['forest_group'].value_counts())


forest_group
medium    312
low       309
high       50
Name: count, dtype: int64


Now let's compute the runoff ratio for every catchment. To do this we need (i) a list of catchment IDs, (ii) a function to load the catchment data and (iii) a function to calculate the runoff ratio. Here is my attempt: 

In [ ]:
def read_camels_data(id): 
    # Read CAMELS-GB data for a given catchment id
    data = pd.read_csv(os.path.join(DATADIR, 'timeseries', f'CAMELS_GB_hydromet_timeseries_{id}_19701001-20150930.csv'), parse_dates=[0])
    return data 

def compute_runoff_ratio(data):
    # Compute runoff ratio for a given catchment dataframe
    data_sum = data.groupby('id')[['precipitation', 'discharge_spec']].sum()
    data_sum['runoff_ratio'] = data_sum['discharge_spec'] / data_sum['precipitation']
    return data_sum

from tqdm import tqdm

# Get a list of catchment IDs 
ids = metadata_lu['gauge_id'].astype(int).tolist()
rr_list = []
for id in tqdm(ids): 
    data = read_camels_data(id)
    data['id'] = id
    rr = compute_runoff_ratio(data)
    rr_list.append(rr)

rr = pd.concat(rr_list)
rr = rr.merge(metadata_lu, how='left', left_on='id', right_on='gauge_id')
print(rr[['gauge_id', 'forest_perc', 'forest_group', 'runoff_ratio']].head())

   gauge_id  forest_perc forest_group  runoff_ratio
0     10002         9.30          low      0.564977
1     10003         7.80          low      0.414309
2      1001        12.78       medium      0.278004
3    101002         6.50          low      0.366922
4    101005         4.72          low      0.257316


In [ ]:
# Optional: enforce ordering
order = ['low', 'medium', 'high']
rr['forest_group'] = pd.Categorical(rr['forest_group'], categories=order, ordered=True)

# There are a few errors in the runoff ratio calculation that lead to values > 1. These can be filtered out.
rr = rr[rr['runoff_ratio'] <= 1.0]

plt.figure(figsize=(6, 4))
rr.boxplot(column='runoff_ratio', by='forest_group', grid=False)

plt.title('Runoff ratio by forest cover group')
plt.suptitle('')  # removes the automatic "Boxplot grouped by ..." title
plt.xlabel('Forest cover group')
plt.ylabel('Runoff ratio (Q/P)')
plt.show()


You can see that on average, catchments with high forest content tends to have a higher average runoff ratio. This is not what we would expect! Generally we would say that, all else being equal, heavily forested catchments would have higher evaporation and hence the runoff ratio would be lower (more evaporation = less discharge). This suggests that forest cover is acting as a proxy for some other control on runoff ratio. In Great Britain, forested catchments tend to be in upland areas in the north and west. Hence they are situated in areas that tend to be wetter on average. I would therefore suggest that climate might be a more dominant control on the runoff ratio than land cover. We will return to this topic in later weeks. 